<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/Model7_%EC%9D%B4%EC%83%81%EC%B9%98_%EC%A0%9C%EA%B1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
r_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_팀타자.csv', encoding='utf-8-sig')
r_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_팀투수.csv', encoding='utf-8-sig')
vs_r_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_상대_팀타자.csv', encoding='utf-8-sig')
vs_r_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_상대_팀투수.csv', encoding='utf-8-sig')
team_data = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig')
sr_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/선발_팀투수_최종.csv', encoding='utf-8-sig')

In [20]:
team_data = team_data[(team_data['득점'] <= 15)]

In [21]:
team_data

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,결과,시즌,경기간간격,월,NC,OB,WO,HT,LG,SS,KT,LT,SK,HH,이닝*3,실점,우취,연장,득점,총득점,총실점,평균득점,평균실점,승수,패배수,승률,피타고리안기대승률,상대총득점,상대총실점,상대평균득점,상대평균실점,상대승률,상대피타고리안기대승률,최근총득점,최근총실점,최근평균득점,최근평균실점,최근승률,최근피타고리안기대승률,총합연봉,평균연봉
306,20160511KTHT0,2016-05-11,HT,KT,0.0,1,W,2016.0,3.0,5.0,0,0,0,1,0,0,0,0,0,0,27,3,0,0,8,146,139,5.034483,4.793103,12,17,0.413793,0.524547,12,16,4.000000,5.333333,0.333333,0.360000,35,29,7.0,5.8,0.4,0.592933,1058700,15801.493
307,20160511KTHT0,2016-05-11,KT,HT,0.0,0,L,2016.0,3.0,5.0,0,0,0,0,0,0,1,0,0,0,24,8,0,0,3,163,170,5.258065,5.483871,15,16,0.483871,0.478988,16,12,5.333333,4.000000,0.666667,0.640000,42,40,8.4,8.0,0.6,0.524376,726300,11173.846
308,20160511NCHH0,2016-05-11,HH,NC,0.0,1,W,2016.0,3.0,5.0,0,0,0,0,0,0,0,0,0,1,27,5,0,0,6,128,221,4.266667,7.366667,8,22,0.266667,0.251192,7,16,2.333333,5.333333,0.333333,0.160656,24,58,4.8,11.6,0.0,0.146193,1454500,19655.405
309,20160511NCHH0,2016-05-11,NC,HH,0.0,0,L,2016.0,3.0,5.0,1,0,0,0,0,0,0,0,0,0,24,6,0,0,5,164,115,5.655172,3.965517,18,11,0.620690,0.670372,16,7,5.333333,2.333333,0.666667,0.839344,51,16,10.2,3.2,1.0,0.910396,872200,13017.910
310,20160511OBSK0,2016-05-11,SK,OB,0.0,1,L,2016.0,1.0,5.0,0,0,0,0,0,0,0,0,1,0,27,7,0,0,3,164,146,4.969697,4.424242,19,14,0.575758,0.557869,14,20,3.500000,5.000000,0.250000,0.328859,43,33,8.6,6.6,0.4,0.629340,955500,14051.471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,1,0,0,27,1,0,0,2,305,320,5.000000,5.245902,29,32,0.475410,0.476014,27,49,3.375000,6.125000,0.250000,0.232907,35,39,7.0,7.8,0.4,0.446103,1067700,16682.812
6396,20200719OBHT0,2020-07-19,HT,OB,0.0,1,L,2020.0,1.0,7.0,0,0,0,1,0,0,0,0,0,0,27,8,0,0,4,307,282,5.032787,4.622951,33,28,0.540984,0.542368,31,43,3.875000,5.375000,0.250000,0.341993,16,23,3.2,4.6,0.4,0.326115,998090,15842.698
6397,20200719OBHT0,2020-07-19,OB,HT,0.0,0,W,2020.0,1.0,7.0,0,1,0,0,0,0,0,0,0,0,27,4,0,0,8,387,348,6.142857,5.523810,37,26,0.587302,0.552912,43,31,5.375000,3.875000,0.750000,0.658007,26,20,5.2,4.0,0.6,0.628253,975000,15476.190
6398,20200719WOSK0,2020-07-19,SK,WO,0.0,1,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,0,1,0,27,3,0,0,4,248,338,3.875000,5.281250,20,44,0.312500,0.349956,38,54,4.750000,6.750000,0.250000,0.331193,29,31,5.8,6.2,0.4,0.466704,935800,15093.548


In [4]:
sr_team_pitcher.columns = ['게임키', '일자', '상대팀코드', '팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '선발_최근 평균자책점', '상대_선발_최근 평균자책점',
       '선발_최근 whip', '상대_선발_최근 whip', '선발_최근 OAVG', '상대_선발_최근 OAVG',
       '선발_최근 OOBP', '상대_선발_최근 OOBP', '선발_최근 OSLG', '상대_선발_최근 OSLG',
       '선발_최근 OOPS', '상대_선발_최근 OOPS']
sr_team_pitcher = sr_team_pitcher[(sr_team_pitcher['실점'] <= 15) ]
r_team_hitter = r_team_hitter.reset_index()
sr_team_pitcher = sr_team_pitcher.reset_index()
del r_team_hitter['index']
del sr_team_pitcher['index']
vs_r_team_hitter = vs_r_team_hitter.reset_index()
sr_team_pitcher = sr_team_pitcher.reset_index()
del vs_r_team_hitter['index']
del sr_team_pitcher['index']
r_team_hitter = pd.merge(r_team_hitter, sr_team_pitcher[['게임키', '팀코드','선발_최근 평균자책점', 
       '선발_최근 whip',  '선발_최근 OAVG', 
       '선발_최근 OOPS', ]], on=['게임키', '팀코드'])
vs_r_team_hitter = pd.merge(vs_r_team_hitter, sr_team_pitcher[['게임키','팀코드', '상대_선발_최근 평균자책점',
       '상대_선발_최근 whip', '상대_선발_최근 OAVG',
        '상대_선발_최근 OOBP',  '상대_선발_최근 OSLG',
       '상대_선발_최근 OOPS']], on=['게임키', '팀코드'])
def createR(row):
    previous = vs_r_team_hitter[(vs_r_team_hitter['시즌'] == row['시즌']) & (vs_r_team_hitter['팀코드'] == row['팀코드']) & (vs_r_team_hitter['상대팀코드'] == row['상대팀코드']) & (vs_r_team_hitter['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근평균득점'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['득점']
    down = len(previous)
    if down == 0:
        row['상대최근평균득점'] = 0
        return row
    row['상대최근평균득점'] = up/down
    return row
vs_r_team_hitter = vs_r_team_hitter.apply(createR, axis=1)

In [ ]:
vs_r_team_hitter

In [5]:
def year_filter(data):
  team_list = []
  for year in range(2016, 2021):
    a = data[(data['일자'] > '{}-05-10'.format(year)) & (data['시즌'] == year)]
    team_list.append(a)
  data = pd.concat(team_list)
  return data

r_team_hitter = year_filter(r_team_hitter)
r_team_pitcher = year_filter(r_team_pitcher)
vs_r_team_pitcher = year_filter(vs_r_team_pitcher)
vs_r_team_hitter = year_filter(vs_r_team_hitter)

In [19]:
team_data = year_filter(team_data)

In [10]:
x_data_h = r_team_hitter[['최근OPS', '최근타율', '최근HR%', '최근K%', '최근BABIP',
       '최근BB%',  '최근wOBA', '최근wRAA',  '최근RC', '최근RC27', '최근wRC', '최근BBK',
       '최근PA', '최근B3B', '최근BB', '최근HR', '최근K',
       '최근HBP', '최근IBB', '최근SB', '최근CS', '최근AB', '최근HIT', '최근B1B', '최근B2B',
       '최근타자득점', '최근득점권타수', '최근득점권안타', '최근득점권타율', '선발_최근 평균자책점', 
       '선발_최근 whip',  '선발_최근 OAVG', 
       '선발_최근 OOPS'
       ]]
#  ,'상대_선발_최근 평균자책점',       '상대_선발_최근 whip', '상대_선발_최근 OAVG',
#         '상대_선발_최근 OOBP',  '상대_선발_최근 OSLG',
#        '상대_선발_최근 OOPS'
x_data_a = vs_r_team_hitter[['상대최근ops', '상대최근타율', 
       '상대최근K%', '상대최근BB%', '상대최근wOBA', '상대최근wRAA', 
       '상대최근HR%',  '상대최근RC', '상대최근RC27', 
       '상대최근PA', '상대최근AB', '상대최근HIT', '상대최근B1B',
       '상대최근B2B', '상대최근B3B', '상대최근BB', '상대최근HR', '상대최근K', '상대최근HBP', '상대최근IBB',
       '상대최근GDP', '상대최근SB', '상대최근CS', '상대최근득점권타수', '상대최근득점권안타', '상대최근득점권타율', '상대최근평균득점',
]]

y_data = r_team_hitter['득점']

In [24]:
x_data_h

,최근OPS,최근타율,최근HR%,최근K%,최근BABIP,최근BB%,최근wOBA,최근wRAA,최근RC,최근RC27,최근wRC,최근BBK,최근PA,최근B3B,최근BB,최근HR,최근K,최근HBP,최근IBB,최근SB,최근CS,최근AB,최근HIT,최근B1B,최근B2B,최근타자득점,최근득점권타수,최근득점권안타,최근득점권타율,선발_최근 평균자책점,선발_최근 whip,선발_최근 OAVG,선발_최근 OOPS
297,0.926,0.322,0.030,0.172,0.366,0.103,0.410,8.809,34.829,7.405,37.258,0.600,37.4,0.4,2.8,1.0,7.2,0.6,0.0,0.2,0.0,32.4,8.8,5.2,2.2,4.2,8.2,1.8,0.219512,9.529412,2.176471,0.368421,1.002571
298,0.909,0.362,0.019,0.175,0.438,0.126,0.424,11.556,36.229,7.953,40.425,0.722,39.0,0.0,5.0,0.8,6.0,0.2,0.0,0.6,0.4,33.2,10.6,8.6,1.2,5.4,10.0,3.4,0.340000,5.965116,1.604651,0.308333,0.833079
299,0.773,0.263,0.035,0.185,0.295,0.110,0.352,-1.835,25.783,5.119,26.193,0.595,40.0,0.0,4.4,1.4,7.4,0.4,0.0,0.8,0.2,35.0,9.2,6.4,1.4,4.8,8.0,1.8,0.225000,2.903226,1.129032,0.211009,0.520996
300,1.033,0.360,0.038,0.137,0.389,0.123,0.452,17.546,41.896,8.837,47.396,0.931,42.6,0.4,5.2,1.6,5.8,0.4,0.2,0.4,0.4,35.6,12.8,8.2,2.6,10.2,11.6,5.4,0.465517,19.636364,3.454545,0.454545,1.213369
301,0.954,0.344,0.035,0.163,0.382,0.101,0.422,12.328,40.333,7.723,44.140,0.622,43.0,0.2,5.4,0.8,6.8,0.6,0.0,0.6,0.6,35.0,11.0,8.2,1.8,5.8,9.2,2.4,0.260870,7.920000,2.160000,0.361905,1.017905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6326,0.849,0.320,0.030,0.146,0.354,0.081,0.383,6.593,29.847,6.152,21.659,0.552,39.6,0.0,3.2,1.2,5.8,0.2,0.0,0.6,0.4,35.6,11.4,8.4,1.8,7.0,9.4,3.2,0.340426,2.554054,1.256757,0.255556,0.623232
6327,0.722,0.272,0.016,0.167,0.320,0.086,0.335,-1.551,20.743,4.342,12.678,0.548,37.4,0.0,3.2,0.6,6.2,0.6,0.2,0.0,0.2,32.4,8.8,6.8,1.4,3.2,8.2,1.4,0.170732,3.471429,1.542857,0.315217,0.740435
6328,0.814,0.325,0.010,0.089,0.344,0.068,0.369,4.051,26.727,5.682,18.584,0.765,38.2,0.4,2.6,0.4,3.4,0.6,0.0,1.6,0.4,33.8,11.0,8.4,1.8,5.2,11.0,3.8,0.345455,6.151899,1.481013,0.315789,0.830130
6329,0.715,0.278,0.025,0.183,0.319,0.081,0.330,-2.609,24.189,4.911,12.381,0.444,39.4,0.0,3.2,1.0,7.2,0.4,0.0,1.4,0.2,35.2,9.8,8.4,0.4,5.8,9.2,3.2,0.347826,4.943662,1.352113,0.255556,0.741111


In [51]:
del team_data['202']
team_data

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,결과,시즌,경기간간격,월,NC,OB,WO,HT,LG,SS,KT,LT,SK,HH,이닝*3,실점,우취,연장,득점,총득점,총실점,평균득점,평균실점,승수,패배수,승률,피타고리안기대승률,상대총득점,상대총실점,상대평균득점,상대평균실점,상대승률,상대피타고리안기대승률,최근총득점,최근총실점,최근평균득점,최근평균실점,최근승률,최근피타고리안기대승률,총합연봉,평균연봉,2016,2017,2018,2019,2020
0,20160511KTHT0,2016-05-11,HT,KT,0.0,1,W,2016.0,3.0,5.0,0,0,0,1,0,0,0,0,0,0,27,3,0,0,8,146,139,5.034483,4.793103,12,17,0.413793,0.524547,12,16,4.000000,5.333333,0.333333,0.360000,35,29,7.0,5.8,0.4,0.592933,1058700,15801.493,1,0,0,0,0
1,20160511KTHT0,2016-05-11,KT,HT,0.0,0,L,2016.0,3.0,5.0,0,0,0,0,0,0,1,0,0,0,24,8,0,0,3,163,170,5.258065,5.483871,15,16,0.483871,0.478988,16,12,5.333333,4.000000,0.666667,0.640000,42,40,8.4,8.0,0.6,0.524376,726300,11173.846,1,0,0,0,0
2,20160511NCHH0,2016-05-11,HH,NC,0.0,1,W,2016.0,3.0,5.0,0,0,0,0,0,0,0,0,0,1,27,5,0,0,6,128,221,4.266667,7.366667,8,22,0.266667,0.251192,7,16,2.333333,5.333333,0.333333,0.160656,24,58,4.8,11.6,0.0,0.146193,1454500,19655.405,1,0,0,0,0
3,20160511NCHH0,2016-05-11,NC,HH,0.0,0,L,2016.0,3.0,5.0,1,0,0,0,0,0,0,0,0,0,24,6,0,0,5,164,115,5.655172,3.965517,18,11,0.620690,0.670372,16,7,5.333333,2.333333,0.666667,0.839344,51,16,10.2,3.2,1.0,0.910396,872200,13017.910,1,0,0,0,0
4,20160511OBSK0,2016-05-11,SK,OB,0.0,1,L,2016.0,1.0,5.0,0,0,0,0,0,0,0,0,1,0,27,7,0,0,3,164,146,4.969697,4.424242,19,14,0.575758,0.557869,14,20,3.500000,5.000000,0.250000,0.328859,43,33,8.6,6.6,0.4,0.629340,955500,14051.471,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4874,20200719LTSS0,2020-07-19,LT,SS,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,1,0,0,27,1,0,0,2,305,320,5.000000,5.245902,29,32,0.475410,0.476014,27,49,3.375000,6.125000,0.250000,0.232907,35,39,7.0,7.8,0.4,0.446103,1067700,16682.812,0,0,0,0,1
4875,20200719OBHT0,2020-07-19,HT,OB,0.0,1,L,2020.0,1.0,7.0,0,0,0,1,0,0,0,0,0,0,27,8,0,0,4,307,282,5.032787,4.622951,33,28,0.540984,0.542368,31,43,3.875000,5.375000,0.250000,0.341993,16,23,3.2,4.6,0.4,0.326115,998090,15842.698,0,0,0,0,1
4876,20200719OBHT0,2020-07-19,OB,HT,0.0,0,W,2020.0,1.0,7.0,0,1,0,0,0,0,0,0,0,0,27,4,0,0,8,387,348,6.142857,5.523810,37,26,0.587302,0.552912,43,31,5.375000,3.875000,0.750000,0.658007,26,20,5.2,4.0,0.6,0.628253,975000,15476.190,0,0,0,0,1
4877,20200719WOSK0,2020-07-19,SK,WO,0.0,1,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,0,1,0,27,3,0,0,4,248,338,3.875000,5.281250,20,44,0.312500,0.349956,38,54,4.750000,6.750000,0.250000,0.331193,29,31,5.8,6.2,0.4,0.466704,935800,15093.548,0,0,0,0,1


In [45]:
seasons = [2016, 2017, 2018, 2019, 2020]
def createSeason(row, season):
  if int(row['시즌']) == season:
    row[str(season)] = 1
  else:
    row[str(season)] = 0
  return row

In [46]:
for s in seasons:
  team_data = team_data.apply(createSeason, axis=1, season=s)

In [ ]:
# def createwSWin(row):
#     previous = team_data[(team_data['시즌'] == row['시즌']) & (team_data['팀코드'] == row['팀코드'])& (team_data['일자'] < row['일자'])]
#     if len(previous) == 0:
#         row['연승'] = 0
#         return row
#     previous_sum = previous.sum()
#     row['wRAA'] = up / down * previous_sum['타자']
#     return row

In [53]:
x_data_h = x_data_h.reset_index()
del x_data_h['index']

In [26]:
team_data = team_data[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '결과', '시즌', '경기간간격', '월',
       'NC', 'OB', 'WO', 'HT', 'LG', 'SS', 'KT', 'LT', 'SK', 'HH', '이닝*3',
       '실점', '우취', '연장', '득점', '총득점', '총실점', '평균득점', '평균실점', '승수', '패배수', '승률',
       '피타고리안기대승률', '상대총득점', '상대총실점', '상대평균득점', '상대평균실점', '상대승률',
       '상대피타고리안기대승률', '최근총득점', '최근총실점', '최근평균득점', '최근평균실점', '최근승률',
       '최근피타고리안기대승률', '총합연봉', '평균연봉']]

Index(['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '결과', '시즌', '경기간간격', '월',
       'NC', 'OB', 'WO', 'HT', 'LG', 'SS', 'KT', 'LT', 'SK', 'HH', '이닝*3',
       '실점', '우취', '연장', '득점', '총득점', '총실점', '평균득점', '평균실점', '승수', '패배수', '승률',
       '피타고리안기대승률', '상대총득점', '상대총실점', '상대평균득점', '상대평균실점', '상대승률',
       '상대피타고리안기대승률', '최근총득점', '최근총실점', '최근평균득점', '최근평균실점', '최근승률',
       '최근피타고리안기대승률', '총합연봉', '평균연봉'],
      dtype='object')

In [54]:
data= pd.concat([x_data_h, team_data], axis=1)
data

,최근OPS,최근타율,최근HR%,최근K%,최근BABIP,최근BB%,최근wOBA,최근wRAA,최근RC,최근RC27,최근wRC,최근BBK,최근PA,최근B3B,최근BB,최근HR,최근K,최근HBP,최근IBB,최근SB,최근CS,최근AB,최근HIT,최근B1B,최근B2B,최근타자득점,최근득점권타수,최근득점권안타,최근득점권타율,선발_최근 평균자책점,선발_최근 whip,선발_최근 OAVG,선발_최근 OOPS,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,결과,...,WO,HT,LG,SS,KT,LT,SK,HH,이닝*3,실점,우취,연장,득점,총득점,총실점,평균득점,평균실점,승수,패배수,승률,피타고리안기대승률,상대총득점,상대총실점,상대평균득점,상대평균실점,상대승률,상대피타고리안기대승률,최근총득점,최근총실점,최근평균득점,최근평균실점,최근승률,최근피타고리안기대승률,총합연봉,평균연봉,2016,2017,2018,2019,2020
0,0.926,0.322,0.030,0.172,0.366,0.103,0.410,8.809,34.829,7.405,37.258,0.600,37.4,0.4,2.8,1.0,7.2,0.6,0.0,0.2,0.0,32.4,8.8,5.2,2.2,4.2,8.2,1.8,0.219512,9.529412,2.176471,0.368421,1.002571,20160511KTHT0,2016-05-11,HT,KT,0.0,1,W,...,0,1,0,0,0,0,0,0,27,3,0,0,8,146,139,5.034483,4.793103,12,17,0.413793,0.524547,12,16,4.000000,5.333333,0.333333,0.360000,35,29,7.0,5.8,0.4,0.592933,1058700,15801.493,1,0,0,0,0
1,0.909,0.362,0.019,0.175,0.438,0.126,0.424,11.556,36.229,7.953,40.425,0.722,39.0,0.0,5.0,0.8,6.0,0.2,0.0,0.6,0.4,33.2,10.6,8.6,1.2,5.4,10.0,3.4,0.340000,5.965116,1.604651,0.308333,0.833079,20160511KTHT0,2016-05-11,KT,HT,0.0,0,L,...,0,0,0,0,1,0,0,0,24,8,0,0,3,163,170,5.258065,5.483871,15,16,0.483871,0.478988,16,12,5.333333,4.000000,0.666667,0.640000,42,40,8.4,8.0,0.6,0.524376,726300,11173.846,1,0,0,0,0
2,0.773,0.263,0.035,0.185,0.295,0.110,0.352,-1.835,25.783,5.119,26.193,0.595,40.0,0.0,4.4,1.4,7.4,0.4,0.0,0.8,0.2,35.0,9.2,6.4,1.4,4.8,8.0,1.8,0.225000,2.903226,1.129032,0.211009,0.520996,20160511NCHH0,2016-05-11,HH,NC,0.0,1,W,...,0,0,0,0,0,0,0,1,27,5,0,0,6,128,221,4.266667,7.366667,8,22,0.266667,0.251192,7,16,2.333333,5.333333,0.333333,0.160656,24,58,4.8,11.6,0.0,0.146193,1454500,19655.405,1,0,0,0,0
3,1.033,0.360,0.038,0.137,0.389,0.123,0.452,17.546,41.896,8.837,47.396,0.931,42.6,0.4,5.2,1.6,5.8,0.4,0.2,0.4,0.4,35.6,12.8,8.2,2.6,10.2,11.6,5.4,0.465517,19.636364,3.454545,0.454545,1.213369,20160511NCHH0,2016-05-11,NC,HH,0.0,0,L,...,0,0,0,0,0,0,0,0,24,6,0,0,5,164,115,5.655172,3.965517,18,11,0.620690,0.670372,16,7,5.333333,2.333333,0.666667,0.839344,51,16,10.2,3.2,1.0,0.910396,872200,13017.910,1,0,0,0,0
4,0.954,0.344,0.035,0.163,0.382,0.101,0.422,12.328,40.333,7.723,44.140,0.622,43.0,0.2,5.4,0.8,6.8,0.6,0.0,0.6,0.6,35.0,11.0,8.2,1.8,5.8,9.2,2.4,0.260870,7.920000,2.160000,0.361905,1.017905,20160511OBSK0,2016-05-11,SK,OB,0.0,1,L,...,0,0,0,0,0,0,1,0,27,7,0,0,3,164,146,4.969697,4.424242,19,14,0.575758,0.557869,14,20,3.500000,5.000000,0.250000,0.328859,43,33,8.6,6.6,0.4,0.629340,955500,14051.471,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4874,0.849,0.320,0.030,0.146,0.354,0.081,0.383,6.593,29.847,6.152,21.659,0.552,39.6,0.0,3.2,1.2,5.8,0.2,0.0,0.6,0.4,35.6,11.4,8.4,1.8,7.0,9.4,3.2,0.340426,2.554054,1.256757,0.255556,0.623232,20200719LTSS0,2020-07-19,LT,SS,0.0,0,W,...,0,0,0,0,0,1,0,0,27,1,0,0,2,305,320,5.000000,5.245902,29,32,0.475410,0.476014,27,49,3.375000,6.125000,0.250000,0.232907,35,39,7.0,7.8,0.4,0.446103,1067700,16682.812,0,0,0,0,1
4875,0.722,0.272,0.016,0.167,0.320,0.086,0.335,-1.551,20.743,4.342,12.678,0.548,37.4,0.0,3.2,0.6,6.2,0.6,0.2,0.0,0.2,32.4,8.8,6.8,1.4,3.2,8.2,1.4,0.170732,3.471429,1.542857,0.315217,0.740435,20200719OBHT0,2020-07-19,HT,OB,0.0,1,L,...,0,1,0,0,0,0,0,0,27,8,0,0,4,307,282,5.032787,4.622951,33,28,0.540984,0.542368,31,43,3.875000,5.375000,0.250000,0.341993,16,23,3.2,4.6,0.4,0.326115,998090,15842.698,0,0,0,0,1
4876,0.814,0.325,0.010,0.089,0.344,0.068,0.369,4.051,26.727,5.682,18.584,0.765,38.2,0.4,2.6,0.4,3.4,0.6,0.0,1.6,0.4,33.8,11.0,8.4,1.8,5.2,11.0,3.8,0.345455,6.151899,1.481013,0.315789,0.830130,20200719OBHT0,2020-07-19,OB,HT,0.0,0,W,...,0,0,0,0,0,0,0,0,27,4,0,0,8,387,348,6.142857,5.523810,37,26,0.587302,0.552912,43,31,5.375000,3.875000,0.750000,0.658007,26,20,5.2,4.0,0.6,0.6

In [56]:
data = data[['최근OPS', '최근타율', '최근HR%', '최근K%', '최근BABIP', '최근BB%', '최근wOBA',
       '최근wRAA', '최근RC', '최근RC27', '최근wRC', '최근BBK', '최근PA', '최근B3B', '최근BB',
       '최근HR', '최근K', '최근HBP', '최근IBB', '최근SB', '최근CS', '최근AB', '최근HIT',
       '최근B1B', '최근B2B', '최근타자득점', '최근득점권타수', '최근득점권안타', '최근득점권타율',
       '선발_최근 평균자책점', '선발_최근 whip', '선발_최근 OAVG', '선발_최근 OOPS', 
      '초말','경기간간격',  'NC', 'OB',
       'WO', 'HT', 'LG', 'SS', 'KT', 'LT', 'SK', 'HH', 
       '2016', '2017', '2018', '2019', '2020']]

In [ ]:
data_des = data.describe()
columns = ['최근OPS', '최근타율', '최근HR%', '최근K%', '최근BABIP',
       '최근BB%',  '최근wOBA', '최근wRAA',  '최근RC', '최근RC27', '최근wRC', '최근BBK',
       '최근PA', '최근B3B', '최근BB', '최근HR', '최근K',
       '최근HBP', '최근IBB', '최근SB', '최근CS', '최근AB', '최근HIT', '최근B1B', '최근B2B',
       '최근타자득점', '최근득점권타수', '최근득점권안타', '최근득점권타율', '선발_최근 평균자책점', 
       '선발_최근 whip',  '선발_최근 OAVG', 
       '선발_최근 OOPS'] 

In [ ]:
mean_list = {}
std_list = {}
for column in columns:
  mean_list[column] = data_des[column].loc['mean']
  std_list[column] = data_des[column].loc['std']

In [ ]:
for column in columns:
  mean = mean_list[column]
  std = std_list[column]
  data = data[( ((data[column] - mean) / std) <= 2.5 ) & ( ((data[column] - mean) / std) >= -2.5 )]

In [ ]:
data.describe()

,최근OPS,최근타율,최근HR%,최근K%,최근BABIP,최근BB%,최근wOBA,최근wRAA,최근RC,최근RC27,최근wRC,최근BBK,최근PA,최근B3B,최근BB,최근HR,최근K,최근HBP,최근IBB,최근SB,최근CS,최근AB,최근HIT,최근B1B,최근B2B,최근타자득점,최근득점권타수,최근득점권안타,최근득점권타율,선발_최근 평균자책점,선발_최근 whip,선발_최근 OAVG,선발_최근 OOPS,득점
count,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000,3573.000000
mean,0.763642,0.277808,0.025003,0.175993,0.318753,0.080344,0.344430,-0.590099,24.702727,5.006208,24.051548,0.488940,38.966919,0.117212,3.128575,0.965911,6.845452,0.545256,0.092639,0.610859,0.287601,34.521746,9.568934,6.814386,1.671425,4.982424,8.920515,2.498517,0.277736,4.906813,1.472782,0.282497,0.773169,5.156171
std,0.096820,0.033836,0.012452,0.032394,0.038050,0.020851,0.037937,6.294679,6.526778,1.364528,8.338673,0.162776,2.073620,0.141769,0.879391,0.491919,1.214505,0.332595,0.132358,0.384811,0.221254,1.704881,1.469544,1.200842,0.581945,1.414761,1.694037,0.806448,0.065125,1.928087,0.282064,0.043770,0.122769,3.367175
min,0.507000,0.185000,0.000000,0.090000,0.218000,0.026000,0.242000,-18.038000,7.717000,1.508000,1.259000,0.125000,33.400000,0.000000,1.000000,0.000000,3.600000,0.000000,0.000000,0.000000,0.000000,30.200000,5.800000,3.600000,0.200000,1.400000,4.400000,0.600000,0.105263,0.385714,0.742574,0.166667,0.442561,0.000000
25%,0.697000,0.254000,0.016000,0.152000,0.292000,0.066000,0.318000,-5.151000,20.099000,4.020000,18.555000,0.371000,37.400000,0.000000,2.400000,0.600000,6.000000,0.200000,0.000000,0.400000,0.200000,33.200000,8.600000,6.000000,1.200000,4.000000,7.600000,1.800000,0.233333,3.494118,1.263158,0.252174,0.683140,3.000000
50%,0.763000,0.278000,0.024000,0.175000,0.319000,0.080000,0.345000,-0.765000,24.373000,4.939000,23.954000,0.471000,39.000000,0.000000,3.000000,1.000000,6.800000,0.600000,0.000000,0.600000,0.200000,34.400000,9.600000,6.800000,1.600000,5.000000,9.000000,2.400000,0.277778,4.680000,1.451613,0.282258,0.770478,5.000000
75%,0.830000,0.302000,0.033000,0.198000,0.345000,0.094000,0.371000,3.571000,29.180000,5.935000,29.744000,0.588000,40.400000,0.200000,3.800000,1.200000,7.600000,0.800000,0.200000,0.800000,0.400000,35.600000,10.600000,7.600000,2.000000,6.000000,10.000000,3.000000,0.322581,6.120000,1.662162,0.312500,0.854748,7.000000
max,1.045000,0.371000,0.059000,0.262000,0.426000,0.138000,0.456000,19.019000,45.139000,9.260000,49.356000,0.968000,45.200000,0.400000,5.600000,2.200000,10.000000,1.400000,0.400000,1.600000,0.800000,39.400000,13.800000,10.200000,3.200000,9.200000,13.800000,4.800000,0.454545,10.968750,2.326531,0.408163,1.130935,15.000000


In [72]:
x_data_h = data[['최근OPS', '최근타율', '최근HR%', '최근K%', '최근BABIP',
       '최근BB%',  '최근wOBA', '최근wRAA',  '최근RC', '최근RC27', '최근wRC', '최근BBK',
       '최근PA', '최근B3B', '최근BB', '최근HR', '최근K',
       '최근HBP', '최근IBB', '최근SB', '최근CS', '최근AB', '최근HIT', '최근B1B', '최근B2B',
       '최근타자득점', '최근득점권타수', '최근득점권안타', '최근득점권타율', '선발_최근 평균자책점', 
       '선발_최근 whip',  '선발_최근 OAVG', 
       '선발_최근 OOPS'
       ]]

# x_data_a = data[['상대최근ops', '상대최근타율', 
#        '상대최근K%', '상대최근BB%', '상대최근wOBA', '상대최근wRAA', 
#        '상대최근HR%',  '상대최근RC', '상대최근RC27', 
#        '상대최근PA', '상대최근AB', '상대최근HIT', '상대최근B1B',
#        '상대최근B2B', '상대최근B3B', '상대최근BB', '상대최근HR', '상대최근K', '상대최근HBP', '상대최근IBB',
#        '상대최근GDP', '상대최근SB', '상대최근CS', '상대최근득점권타수', '상대최근득점권안타', '상대최근득점권타율', '상대최근평균득점',
# ]]

y_data = data['득점']

KeyError: ignored

In [73]:
data

,최근OPS,최근타율,최근HR%,최근K%,최근BABIP,최근BB%,최근wOBA,최근wRAA,최근RC,최근RC27,최근wRC,최근BBK,최근PA,최근B3B,최근BB,최근HR,최근K,최근HBP,최근IBB,최근SB,최근CS,최근AB,최근HIT,최근B1B,최근B2B,최근타자득점,최근득점권타수,최근득점권안타,최근득점권타율,선발_최근 평균자책점,선발_최근 whip,선발_최근 OAVG,선발_최근 OOPS,초말,경기간간격,NC,OB,WO,HT,LG,SS,KT,LT,SK,HH,2016,2017,2018,2019,2020
0,0.926,0.322,0.030,0.172,0.366,0.103,0.410,8.809,34.829,7.405,37.258,0.600,37.4,0.4,2.8,1.0,7.2,0.6,0.0,0.2,0.0,32.4,8.8,5.2,2.2,4.2,8.2,1.8,0.219512,9.529412,2.176471,0.368421,1.002571,1,3.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,0.909,0.362,0.019,0.175,0.438,0.126,0.424,11.556,36.229,7.953,40.425,0.722,39.0,0.0,5.0,0.8,6.0,0.2,0.0,0.6,0.4,33.2,10.6,8.6,1.2,5.4,10.0,3.4,0.340000,5.965116,1.604651,0.308333,0.833079,0,3.0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0.773,0.263,0.035,0.185,0.295,0.110,0.352,-1.835,25.783,5.119,26.193,0.595,40.0,0.0,4.4,1.4,7.4,0.4,0.0,0.8,0.2,35.0,9.2,6.4,1.4,4.8,8.0,1.8,0.225000,2.903226,1.129032,0.211009,0.520996,1,3.0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,1.033,0.360,0.038,0.137,0.389,0.123,0.452,17.546,41.896,8.837,47.396,0.931,42.6,0.4,5.2,1.6,5.8,0.4,0.2,0.4,0.4,35.6,12.8,8.2,2.6,10.2,11.6,5.4,0.465517,19.636364,3.454545,0.454545,1.213369,0,3.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0.954,0.344,0.035,0.163,0.382,0.101,0.422,12.328,40.333,7.723,44.140,0.622,43.0,0.2,5.4,0.8,6.8,0.6,0.0,0.6,0.6,35.0,11.0,8.2,1.8,5.8,9.2,2.4,0.260870,7.920000,2.160000,0.361905,1.017905,1,1.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4874,0.849,0.320,0.030,0.146,0.354,0.081,0.383,6.593,29.847,6.152,21.659,0.552,39.6,0.0,3.2,1.2,5.8,0.2,0.0,0.6,0.4,35.6,11.4,8.4,1.8,7.0,9.4,3.2,0.340426,2.554054,1.256757,0.255556,0.623232,0,1.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4875,0.722,0.272,0.016,0.167,0.320,0.086,0.335,-1.551,20.743,4.342,12.678,0.548,37.4,0.0,3.2,0.6,6.2,0.6,0.2,0.0,0.2,32.4,8.8,6.8,1.4,3.2,8.2,1.4,0.170732,3.471429,1.542857,0.315217,0.740435,1,1.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4876,0.814,0.325,0.010,0.089,0.344,0.068,0.369,4.051,26.727,5.682,18.584,0.765,38.2,0.4,2.6,0.4,3.4,0.6,0.0,1.6,0.4,33.8,11.0,8.4,1.8,5.2,11.0,3.8,0.345455,6.151899,1.481013,0.315789,0.830130,0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4877,0.715,0.278,0.025,0.183,0.319,0.081,0.330,-2.609,24.189,4.911,12.381,0.444,39.4,0.0,3.2,1.0,7.2,0.4,0.0,1.4,0.2,35.2,9.8,8.4,0.4,5.8,9.2,3.2,0.347826,4.943662,1.352113,0.255556,0.741111,1,1.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [75]:
del y_data['level_0']
y_data

,득점
0,8
1,3
2,6
3,5
4,3
...,...
4874,2
4875,4
4876,8
4877,4


In [64]:
y_data = y_data.reset_index()
del y_data['index']

In [12]:
from sklearn.model_selection import train_test_split
x_train_h, x_test_h, y_train, y_test = train_test_split(
    x_data_h, y_data, test_size = 0.25 
)

In [76]:
from sklearn.model_selection import train_test_split
x_train_h, x_test_h, y_train, y_test = train_test_split(
    data, y_data, test_size = 0.25 
)

In [77]:
x_train_h

,최근OPS,최근타율,최근HR%,최근K%,최근BABIP,최근BB%,최근wOBA,최근wRAA,최근RC,최근RC27,최근wRC,최근BBK,최근PA,최근B3B,최근BB,최근HR,최근K,최근HBP,최근IBB,최근SB,최근CS,최근AB,최근HIT,최근B1B,최근B2B,최근타자득점,최근득점권타수,최근득점권안타,최근득점권타율,선발_최근 평균자책점,선발_최근 whip,선발_최근 OAVG,선발_최근 OOPS,초말,경기간간격,NC,OB,WO,HT,LG,SS,KT,LT,SK,HH,2016,2017,2018,2019,2020
4745,0.729,0.277,0.015,0.143,0.317,0.097,0.342,-0.540,23.965,4.939,14.374,0.679,39.2,0.0,3.8,0.6,5.6,0.6,0.0,0.0,0.0,34.6,9.6,7.6,1.4,5.0,10.2,3.6,0.352941,4.447059,1.376471,0.250000,0.654653,1,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1163,0.730,0.270,0.021,0.200,0.328,0.067,0.334,-2.921,22.272,4.422,23.414,0.333,39.0,0.0,2.6,0.8,7.8,1.4,0.0,0.8,0.2,34.8,9.4,7.0,1.6,3.4,9.2,1.6,0.173913,2.934783,0.815217,0.187500,0.510714,1,2.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2232,0.713,0.269,0.025,0.197,0.317,0.040,0.314,-6.737,22.380,4.255,21.079,0.205,39.6,0.2,1.6,1.0,7.8,0.6,0.0,0.0,0.6,37.2,10.0,7.2,1.6,4.4,8.4,2.8,0.333333,5.192308,1.576923,0.295238,0.832712,1,2.0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2409,0.921,0.333,0.029,0.146,0.372,0.083,0.401,9.631,36.752,7.875,38.711,0.633,41.4,0.2,3.4,1.2,6.0,1.0,0.4,0.4,0.4,36.0,12.0,8.2,2.4,9.4,9.8,4.6,0.469388,5.400000,1.552941,0.313559,0.879154,1,2.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
721,0.932,0.317,0.047,0.174,0.345,0.094,0.411,9.485,38.063,7.727,39.335,0.541,42.6,0.4,4.0,2.0,7.4,1.2,0.0,0.4,0.0,36.6,11.6,8.0,1.2,8.2,10.4,3.0,0.288462,4.558442,1.285714,0.272727,0.746383,1,1.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,0.841,0.323,0.014,0.152,0.372,0.081,0.382,3.877,33.311,6.762,33.447,0.531,39.8,0.2,3.2,0.8,6.0,0.2,0.0,0.4,0.0,35.2,11.0,7.2,2.8,6.4,9.4,3.2,0.340426,6.120000,1.520000,0.294118,0.843224,1,1.0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4525,0.668,0.235,0.022,0.230,0.289,0.071,0.305,-6.311,16.285,3.209,7.613,0.310,36.6,0.2,2.6,0.8,8.4,0.6,0.0,0.0,0.6,33.2,7.8,5.2,1.6,4.4,6.0,2.2,0.366667,6.287671,1.561644,0.270833,0.805060,0,1.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
763,0.622,0.195,0.016,0.219,0.246,0.135,0.299,-11.178,15.349,2.981,16.009,0.667,38.8,0.0,5.2,0.6,8.4,0.6,0.4,1.4,0.2,31.8,6.2,4.2,1.4,2.6,8.0,1.0,0.125000,4.714286,1.809524,0.292135,0.747024,1,1.0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
3439,0.647,0.244,0.011,0.200,0.296,0.083,0.301,-4.929,16.701,3.469,16.265,0.417,36.0,0.4,3.0,0.4,7.2,0.0,0.0,0.4,0.2,32.0,7.8,5.8,1.2,3.0,6.6,2.0,0.303030,1.968750,1.125000,0.252174,0.582957,0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [78]:
y_train

,득점
4745,2
1163,6
2232,4
2409,7
721,8
...,...
711,7
4525,4
763,9
3439,4


In [81]:
# 회귀 일반
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU
from keras import metrics, optimizers

model_h = Sequential()
model_h.add(Dense(32, input_shape=(50, ), kernel_initializer='he_normal'))
model_h.add(LeakyReLU(alpha=0.1))
model_h.add(Dropout(0.3))  
model_h.add(Dense(32, kernel_initializer='he_normal'))
model_h.add(LeakyReLU(alpha=0.1))
model_h.add(Dropout(0.1))  
model_h.add(Dense(32, kernel_initializer='he_normal'))
model_h.add(LeakyReLU(alpha=0.1))
model_h.add(Dropout(0.1))  
model_h.add(Dense(1))
model_h.add(LeakyReLU(alpha=0.1))

adam = optimizers.Adam(lr = 0.005) 

model_h.compile(loss='mse', optimizer = 'adam', metrics=['mse'])

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=20)
model_h.fit(x_train_h, y_train, validation_split=0.2, verbose = 1,
          batch_size=1, epochs=100, callbacks = [early_stopping])

loss, accuracy = model_h.evaluate(x_test_h, y_test, batch_size=1)
print('lose: ', loss)
print('mse: ', accuracy)

y_pred = model_h.predict(x_test_h)

Epoch 1/100
2927/2927 [==============================] - 4s 1ms/step - loss: 21.1601 - mse: 21.1601 - val_loss: 13.5510 - val_mse: 13.5510
Epoch 2/100
2927/2927 [==============================] - 4s 1ms/step - loss: 13.0231 - mse: 13.0231 - val_loss: 11.0006 - val_mse: 11.0006
Epoch 3/100
2927/2927 [==============================] - 4s 1ms/step - loss: 12.4977 - mse: 12.4977 - val_loss: 10.6272 - val_mse: 10.6272
Epoch 4/100
2927/2927 [==============================] - 4s 1ms/step - loss: 12.2870 - mse: 12.2870 - val_loss: 10.9168 - val_mse: 10.9168
Epoch 5/100
2927/2927 [==============================] - 4s 1ms/step - loss: 12.0253 - mse: 12.0253 - val_loss: 10.9948 - val_mse: 10.9948
Epoch 6/100
2927/2927 [==============================] - 4s 1ms/step - loss: 12.1151 - mse: 12.1151 - val_loss: 11.7928 - val_mse: 11.7928
Epoch 7/100
2927/2927 [==============================] - 4s 1ms/step - loss: 11.8694 - mse: 11.8694 - val_loss: 11.6137 - val_mse: 11.6137
Epoch 8/100
2927/2927 [====

In [82]:
for i in range(len(y_pred)):
  print(y_pred[i], y_test.values[i])

[5.2871084] [3]
[4.6715956] [6]
[5.3746986] [5]
[5.478239] [12]
[4.9121757] [13]
[4.6938124] [6]
[4.9749703] [11]
[4.337376] [7]
[5.0111046] [2]
[5.3253794] [8]
[5.0983405] [4]
[5.0057487] [2]
[4.4790945] [11]
[5.225967] [11]
[4.4236655] [1]
[5.2127213] [9]
[5.5071526] [12]
[4.739946] [12]
[4.3767467] [4]
[4.9219775] [5]
[5.145737] [5]
[5.5334435] [4]
[4.954735] [5]
[4.8337827] [2]
[5.1159306] [2]
[5.246926] [15]
[5.4156876] [2]
[5.51552] [13]
[4.42506] [3]
[5.272106] [10]
[5.1899548] [6]
[4.9727044] [9]
[4.3897] [3]
[5.0088196] [15]
[5.093176] [1]
[4.701672] [1]
[4.597392] [2]
[4.32799] [11]
[5.44335] [5]
[5.165761] [1]
[4.6170015] [1]
[5.033985] [8]
[5.1817884] [14]
[5.383337] [9]
[4.290746] [4]
[5.205326] [7]
[5.163286] [5]
[5.369629] [6]
[5.864386] [9]
[4.750788] [4]
[5.2846227] [4]
[5.025821] [5]
[5.1708517] [4]
[5.3075004] [2]
[4.3515058] [5]
[5.021209] [4]
[5.011945] [10]
[4.3136926] [12]
[4.9672604] [3]
[5.1705523] [2]
[5.016091] [12]
[5.853241] [9]
[5.8101015] [13]
[4.60001] [